<a href="https://colab.research.google.com/github/ssv273/Neural_Univesity/blob/main/hw_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Полносвязная нейронная сеть**

*Разбор данного раздела:* https://youtu.be/lP2k--9i7Ug?t=139



In [1]:
import tensorflow as tf #Импортируем tensorflow
import datetime, os #Для подсчета времени и работы с файловой системой
import numpy as np #Для работы с матрицами 
from tensorflow.keras import utils #Для работы с категориальными данными 
import sys #Для специльного вывода
import matplotlib.pyplot as plt #Для виузализации 
import random #Для генерации случайных чисел 
from tqdm.notebook import tqdm

**Универсальная функция для формирования датасета**

*Разбор данного раздела:* https://youtu.be/lP2k--9i7Ug?t=167



In [2]:
#Функция формирования датсета mnist 
#flatten - вытягивание картинки из формы 28, 28 в 784
def data_mnist(flatten = True):
  #Скачивааем датасет 
  (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    
  if flatten:#Для полносвязной сети  
    X_train = X_train.reshape(-1, 784)
    X_test = X_test.reshape(-1, 784)
  else: #Для сверточной сети 
    X_train = X_train.reshape(-1, 28, 28, 1)
    X_test = X_test.reshape(-1, 28, 28, 1)
  #Первеодим во float
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  #Переводим все значения в диапазон от 0 до 1
  X_train /= 255
  X_test /= 255

  print('Форма x_train:', X_train.shape)
  print(X_train.shape[0], 'обучающих примеров')
  print(X_test.shape[0], 'проверочных примеров')
  #Переводим в OneHot
  y_train = utils.to_categorical(y_train, 10).astype(np.float32)
  y_test = utils.to_categorical(y_test, 10).astype(np.float32)
  #Возвращаем датасет
  return X_train, y_train, X_test, y_test #Возвращаем датасет 

*Разбор данного раздела:* https://youtu.be/lP2k--9i7Ug?t=648

**Загрузка данных**



In [3]:
x_train, y_train, x_test, y_test = data_mnist() #Загружаем датасет 

Форма x_train: (60000, 784)
60000 обучающих примеров
10000 проверочных примеров


**Параметры обучения и оптимизации**

In [4]:
learningRate = 0.5
epochs = 10
batchSize = 100

**Тренируемые параметры**



In [5]:
trainableParams = [] #Лист тренируемых параметров
# Объявляем веса, W1
trainableParams.append(tf.Variable(tf.random.normal([784, 300], stddev=0.03), name='W1')) #300 размер скрытого слоя, инциалзириуем значения

# Используя нормальное распределение со средним ноль и статистическим отклонением 0.03, определяем bias(Аналогичная есть у numpy)
trainableParams.append(tf.Variable(tf.random.normal([300], stddev=0.03), name='b1'))

# То же делаем для весов и bias(отклонения) от скрытого к выходному
trainableParams.append(tf.Variable(tf.random.normal([300, 10], stddev=0.03), name='W2'))
trainableParams.append(tf.Variable(tf.random.normal([10]), name='b2'))

In [6]:
#Функция подсчета ошибки
def loss(pred , target):
    return tf.losses.categorical_crossentropy(target , pred)

In [7]:
#Полносвязный слой с несколькими функциями активации
def dense(x , params):
    W1 = params[0]
    b1 = params[1]
    W2 = params[2]
    b2 = params[3]

    #a @ b - аналог tf.matmul(a, b)
    hiddenOut = tf.nn.relu(x@W1+b1) #Умножаем вход на веса W1, прибавляем b1 и применяем функцию активации relu
    y = tf.nn.softmax(hiddenOut@W2+b2) #Умножаем выход скрытого слоя на веса W2, прибавляем b2 и применяем функцию активации softmax
    return y

In [8]:
#Модель 
def model(x):
    y = dense(x, trainableParams)
    return y 


In [9]:
#Посмотрим, какой выход у модели, если на вход подать случайную картинку 
model(x_train[[1]]) #Смотрим вывод сети для первого изображения из MNIST

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.08538685, 0.3148806 , 0.0327628 , 0.05977216, 0.09715238,
        0.08591279, 0.12750764, 0.05598448, 0.11360694, 0.0270333 ]],
      dtype=float32)>

In [10]:
#Функция для информативного вывода обучения
#На вход получает:
# current - номер текущего батча
# amount - число всех батчей 
# params - словарь дополнительных параметров для вывода 

def print_log(current, amount, params):
  #Мы будем выводить прогрессбар, подобный прогрессбару при обучении нейронных сетей на Keras
  #Формат прогрессбара 23/120 [=====>------------------------]
  
  bar_len = 30 #Длина бара 
  percent = int(bar_len * current / amount) #Процент выполненной работы
  progressbar = ''

  for i in range(bar_len): #Проходим по всем элементам прогрессбара и добавляем символы в соответсвии с прогресом 
    if(i < percent):
      progressbar += '='
    elif(i == percent):
      progressbar += '>'
    else:
      progressbar += '-'

  #Добавялем в финальное сообщение символ переноса коретки консоли на начальную строку, добавляпем информацию о номере батча
  #количестве всех батчей, прогрессбар
  #Символ переноса коретки \r добавляется для того, чтобы каждый новый батч перезаписывать вывод. Таким образом вывод не будет засоряться повторяющейся информацией
  message = "\r" + str(current) + '/' + str(amount) + ' [' + progressbar + ']  ' 
  #Добавляем дополнительные параметры в вывод
  for key in params:
    message += key + str(params[key]) + '. '
  
  #Выводим информацию аналогом print. Используется именно эта функция, так как она позволяет осуществлять перенос каретки 
  sys.stdout.write(message)
  #Очищаем буфер вывода
  sys.stdout.flush()

In [11]:
optimizer = tf.keras.optimizers.SGD(learning_rate=learningRate) #Задаем оптимизатор 
m = tf.keras.metrics.Accuracy() #Задаем метрику

def train(model, inputs, outputs): #Функция тренировки сети 

    with tf.GradientTape() as tape:
      current_loss = tf.reduce_mean(loss(model(inputs), outputs)) #Считаем ошибку

      # Градиентный спуск. Инициализируем через learning rate
      # Функция реализует градиентный спуск и обратное распространение ошибки.
      grads = tape.gradient(current_loss , trainableParams)
      #Применение градиентного спуска
      optimizer.apply_gradients(zip(grads , trainableParams))
      #Подсчет точности сети 
      _ = m.update_state(np.argmax(outputs, axis=1), np.argmax(model(inputs), axis=1))
    return current_loss, m.result().numpy()

In [12]:
!rm -R /tmp/mylogs/eager #Очищаем папку с записанной информацией обучения (если код запускается не в первый раз)

In [13]:
path = '/tmp/mylogs/eager/' #Указываем путь для сохранения даных для Tensorboard

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") #Получаем информацию о текущем времени для добавления данной информации к имени  
loss_log_dir = path + current_time + '/data' #Создаем папку с именем из текущего времени
loss_summary_writer = tf.summary.create_file_writer(loss_log_dir) #Создаем средство записи файла резюме для данного каталога

amount_bathces = int(len(x_train) / batchSize) #Считаем число батчей для каждой эпохи. Понадобится для вывода

with loss_summary_writer.as_default(): #Используя созданное средство для записи файла в резюме 
 
  for epoch in range(1, epochs + 1): #Проходим по каждой эпохе
    learningEpochStartTime = datetime.datetime.now() #Запоминаем время начала эпохи 
    # print('Эпоха', epoch , '/', epochs) #Пишем текущую эпоху и общее число эпох
    avg_loss = 0 #Задаем среднюю ошибку

    for batch in tqdm(range(0, len(x_train), batchSize), desc=f'Эпоха {epoch}/{epochs}', bar_format='{l_bar}{bar}{elapsed_s:.0f} c'): #Проходим по x_train с шагом batchSize
      current_loss, accuracy = train(model, x_train[batch: batch + batchSize], y_train[batch: batch + batchSize]) #Тренируем сеть и получаем значение ошибки
      avg_loss += current_loss #Считаем среднюю ошибку

      #Задаем параметры, которые будем выводить 
      params = {'Время обучения на эпохе: ': datetime.datetime.now() - learningEpochStartTime, #Считаем время обучения на данной эпохе и добавляем в словарь
                'loss: ': round(current_loss.numpy(), 4), #Переводим ошибку в Numpy и добавляем в словарь
                'accuracy: ': round(accuracy, 4)} #Добавляем точность в словарь
      if(batch >= len(x_train) - batchSize): #На последнем батче
        params['loss: '] = round((avg_loss / amount_bathces).numpy(), 4) #Выводим среднюю ошибку на всей эпохе

      current_batch = int(batch / batchSize) + 1 #Считаем номер текущего батча
    #   print_log(current_batch, amount_bathces, params) #Выводим всю нужную информацию 
    print(f"loss : {params['loss: ']:.4f}              accuracy: {params['accuracy: ']:.4f}")
    tf.summary.scalar("avg_loss", avg_loss, step=epoch) #Сохраняем данные для Tensorboard
    tf.summary.scalar("accuracy", accuracy, step=epoch) #Сохраняем данные для Tensorboard
    loss_summary_writer.flush() #Очищаем буфер вывода 
    print() #Вручную переносим каретку на следующую строку, чтобы не стирать финальные значения сети на эпохе

Эпоха 1/10:   0%|          0 c

loss : 0.2938              accuracy: 0.9528



Эпоха 2/10:   0%|          0 c

loss : 0.1150              accuracy: 0.9718



Эпоха 3/10:   0%|          0 c

...

loss : 0.0209              accuracy: 0.9919



Эпоха 9/10:   0%|          0 c

loss : 0.0166              accuracy: 0.9928



Эпоха 10/10:   0%|          0 c

loss : 0.0133              accuracy: 0.9935

